In [1]:
import cPickle as pickle
import mxnet
from data_utilities import load_mnist
import mx_layers as layers
from mx_utility import mxnet_array_mapping

In [2]:
stretched = load_mnist('stretched_mnist/', scale=1.0, shape=(1, 56, 56))
stretched_canvas = load_mnist('stretched_canvas_mnist/', scale=1.0, shape=(1, 56, 56))

In [3]:
n_residual_layers = 3
postfix = 'round0'
identifier = 'shrinked-mnist-residual-network-%d-%s' % (n_residual_layers, postfix)
parameters, states = pickle.load(open('parameters/%s' % identifier, 'rb'))

In [4]:
def _normalized_convolution(**args):
  network = layers.convolution(**args)
  network = layers.batch_normalization(network)
  network = layers.ReLU(network)
  return network

In [5]:
network = layers.variable('data')

for index in range(3):
  network = _normalized_convolution(X=network, n_filters=16, kernel_shape=(5, 5), stride=(1, 1), pad=(2, 2))
  network = layers.pooling(X=network, mode='maximum', kernel_shape=(2, 2), stride=(2, 2), pad=(0, 0))

shared_weight = layers.variable('shared_convolution_weight')
shared_bias = layers.variable('shared_convolution_bias')
kwargs = {'n_filters' : 16, 'kernel_shape' : (3, 3), 'stride' : (1, 1), 'pad' : (1, 1)}

for index in range(n_residual_layers):
  identity = network
  residual = _normalized_convolution(X=network, weight=shared_weight, bias=shared_bias, **kwargs)
  network = identity + residual

network = layers.pooling(X=network, mode='average', global_pool=True, kernel_shape=(1, 1), stride=(1, 1), pad=(0, 0))
network = layers.flatten(network)
network = layers.fully_connected(X=network, n_hidden_units=10)
# network = layers.softmax_loss(prediction=network, normalization='batch', id='softmax')

In [6]:
N = 64
args = {'data' : stretched[0][:N]}
args.update(parameters)
args = mxnet_array_mapping(args)
aux_states = mxnet_array_mapping(states)
executor = network.bind(mxnet.cpu(), args, aux_states=aux_states)

In [7]:
for key in parameters.keys():
    if 'convolution' in key: print key

convolution1_weight
shared_convolution_bias
convolution0_bias
convolution1_bias
convolution0_weight
convolution2_bias
convolution2_weight
shared_convolution_weight


In [8]:
help(network.bind)

Help on method bind in module mxnet.symbol:

bind(self, ctx, args, args_grad=None, grad_req='write', aux_states=None, group2ctx=None, shared_exec=None) method of mxnet.symbol.Symbol instance
    Bind current symbol to get an executor.
    
    Parameters
    ----------
    ctx : Context
        The device context the generated executor to run on.
    
    args : list of NDArray or dict of str to NDArray
        Input arguments to the symbol.
    
        - If type is list of NDArray, the position is in the same order of list_arguments.
        - If type is dict of str to NDArray, then it maps the name of arguments
          to the corresponding NDArray.
        - In either case, all the arguments must be provided.
    
    args_grad : list of NDArray or dict of str to NDArray, optional
        When specified, args_grad provide NDArrays to hold
        the result of gradient value in backward.
    
        - If type is list of NDArray, the position is in the same order of list_arguments